In [73]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import random
from tqdm.auto import tqdm
import math

logging.basicConfig(level=logging.DEBUG)

In [74]:
def load_data(city):
    global CITIES, DIST_MATRIX
    CITIES = pd.read_csv(f'cities/{city}.csv', header=None, names=['name', 'lat', 'lon']) # contains all cities with its lat and lon
    DIST_MATRIX = np.zeros((len(CITIES), len(CITIES))) # quadratice matrix of distances between cities, initialized with all zeros
    for c1, c2 in combinations(CITIES.itertuples(), 2): # for each pair of cities
        DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
            (c1.lat, c1.lon), (c2.lat, c2.lon)
        ).km # compute the distance between lat and lon
    # CITIES.head() # print the first five rows

In [75]:
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1] # check if the start city is equal to the end one
    assert set(tsp) == set(range(len(CITIES))) # check if the tsp include all the cities almost one time

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]): # for each pair of cities
        tot_cost += DIST_MATRIX[c1, c2] # compute the total cost as the total cost plus the distance between the two cities
    return tot_cost

In [76]:
MAX_GENERATIONS = 5_000
POPULATION_SIZE = 10
RATE = .5 # used for Order Crossover
MUT_RATE = .1 # used for mutation

### Greedy Algorithm

In [77]:
def greedy_algorithm():
    visited = np.full(len(CITIES), False)
    dist = DIST_MATRIX.copy()
    city = 0
    visited[city] = True
    tsp = list()
    tsp.append(int(city))
    while not np.all(visited):
        dist[:, city] = np.inf
        closest = np.argmin(dist[city])
        visited[closest] = True
        city = closest
        tsp.append(int(city))
    tsp.append(tsp[0])

    logging.info(f"result Greedy Algorithm: Found a path of {len(tsp)-1} steps, total length {tsp_cost(tsp):.2f}km")
    return tsp

### Utility Functions

In [78]:
def xover(p1, p2):
    assert len(p1) == len(p2)
    child = np.full(len(p1), -1) # creating a child with all -1 inside of len p1
    start, end = sorted(random.sample(range(1, len(p1)), 2)) # selecting two rand indexes to copy elements from p1
    
    child[start:end] = p1[start:end] # copying the elements from p1 to the child
    c_pos = (end + 1) % len(p1) # updating the c_pos as end + 1, if end+1 = len(p1) then c_pos = 0
    
    for city in p2:
        if city not in child:
            if c_pos == start:
                c_pos = (c_pos + (end - start)) % len(p1)
            child[c_pos] = city
            c_pos = (c_pos + 1) % len(p1)
            
    # there is a -1 in the child due to the fact that a city cannot compare two times in child
    if -1 in child:
        child[child == -1] = child[0] # change elem=-1 to child[0] (the start city)
                
    i = len(child) - 1 - np.where(child[::-1] == child[0])[0][0] # finding the index of the last occurency of the first city

    if i != len(child) - 1:  # if the element is not in the last position move it
        e = child[i]
        child = np.delete(child, i)  # remove the element
        child = np.append(child, e)  # add the element at the end
    
    return child

def mutation(tsp):
    x = 0
    while x < MUT_RATE:
        i, j = random.sample(range(1, len(tsp) - 1), 2) # finding two indexes, don't taking the first and the last
        tsp[i], tsp[j] = tsp[j], tsp[i]
        x = random.random()
    return tsp

def inver_over_xover(tsp, population, rate = RATE):
    tsp_c = tsp.copy()
    for i in range(1, len(tsp_c) - 1):
        if random.random() < rate:
            target = tsp_c[i]
            if random.random() < .5:
                # select another city from the tsp
                other_t = random.choice(tsp_c[1:-1])
            else:
                # select another city from the population
                rand_tsp = random.choice(population)
                while True:
                    other_t = random.choice(rand_tsp[1:-1])
                    # take the city only if it is different from the first city in tsp
                    if other_t != tsp_c[0]:
                        break
            # finding indexes of the cities
            i_target = np.where(tsp_c == target)[0][0]
            i_other = np.where(tsp_c == other_t)[0][0]
            # swapping path between the two cities
            if i_target < i_other:
                tsp_c[i_target:i_other+1] = list(reversed(tsp_c[i_target:i_other+1]))
            else:
                tsp_c[i_other:i_target+1] = list(reversed(tsp_c[i_other:i_target+1]))
    return tsp_c

def compute_total_dist(tsp):
    # computing the total distance of the tsp, between the cities
    return sum(DIST_MATRIX[tsp[i], tsp[i+1]] for i in range(len(DIST_MATRIX) - 1)) + DIST_MATRIX[tsp[-1], tsp[0]]

### Evolutionary Algorithm - Inverse Crossover

In [79]:
def EA_ioc(tsp):
    population = [tsp]
    for _ in range(100):
        path = np.random.permutation(len(tsp) - 1).tolist()
        path.append(path[0])
        population.append(path)
        
    for _ in tqdm(range(MAX_GENERATIONS)):
        population.sort(key=tsp_cost)
        new_pop = population[:POPULATION_SIZE]
        
        while len(new_pop) < len(population):
            p1, p2 = random.sample(new_pop, 2)
            child = xover(p1, p2)
            child = inver_over_xover(child, population, rate = RATE)
            new_pop.append(child)
            
        population = new_pop

    min_tsp = min(population, key=tsp_cost)
    logging.info(f"result Evolutionary Algorithm - Inverse Crossover: Found a path of {len(min_tsp)-1} steps, total length {tsp_cost(min_tsp):.2f}km")

### Evolutionary Algorithm - Order Crossover

In [80]:
def EA_oc(tsp):
    population = [tsp]
    for _ in range(100):
        path = np.random.permutation(len(tsp) - 1).tolist()
        path.append(path[0])
        population.append(path)
        
    for _ in tqdm(range(MAX_GENERATIONS)):
        population.sort(key=tsp_cost)
        new_pop = population[:POPULATION_SIZE]
        
        while len(new_pop) < len(population):
            p1, p2 = random.sample(new_pop, 2)
            child = xover(p1, p2)
            
            if np.random.random() < RATE:
                child = mutation(child)
            new_pop.append(child)
            
        population = new_pop

    min_tsp = min(population, key=tsp_cost)
    logging.info(f"result Evolutionary Algorithm - Order Crossover: Found a path of {len(min_tsp)-1} steps, total length {tsp_cost(min_tsp):.2f}km")


### Simulated Annealing

In [81]:
def simulated_annealing(temperature = 100, cooling_factor = .99):
    # starting with a random path
    path = list(range(len(CITIES)))
    random.shuffle(path)
    c_dist = compute_total_dist(path)
    best_dist = c_dist
    best_path = path

    while temperature > .01:
        for _ in range(MAX_GENERATIONS):
            new_path = path.copy()
            i, j = sorted(random.sample(range(len(CITIES)), 2))
            new_path[i:j+1] = reversed(path[i:j+1])
            new_dist = compute_total_dist(new_path)
            # computing the ratio to accept the new path
            dist_ratio = new_dist - c_dist

            if dist_ratio < 0 or random.random() < math.exp(-dist_ratio / temperature): # low dist or high temperature means to accept a worse path
                path = new_path
                c_dist = new_dist
                if c_dist < best_dist:
                    best_dist = c_dist
                    best_path = path
        # updating the temperature using the cooling factor
        temperature *= cooling_factor

    # return the best path adding the first city at the end
    tsp = best_path + [best_path[0]]
    logging.info(f"result Simulated Annealing: Found a path of {len(tsp)-1} steps, total length {tsp_cost(tsp):.2f}km")

In [ ]:
cities = ['italy', 'russia', 'vanuatu', 'china', 'us']
for city in cities:
    logging.info(f"Running algorithms for {city}")
    # loading all the data
    load_data(city)
    # --------------------- Running Algorithms ---------------------
    tsp = greedy_algorithm()
    EA_ioc(tsp)
    EA_oc(tsp)
    simulated_annealing()